Instructions : Air management system

Your goal is to build an airplanes traffic management system.

Details
Your program should rely on four classes: Airline, Airplane, Flight and Airport.

Consider every plane can fly only once per day.

The Airline class

Attributes:

id (str) A two letters code

name (str)

planes : A list of Airplanes belonging to this airline (see below the Airplane class)

This class has no methods

The Airplane class

Attributes:

id (int)

current_location : The Airport where the airplane is currently located (see below the Airport class)

company : The airline this airplane belongs to (see above the Airline class)

next_flights : The list of Flights. Every future flights of the airplane, this list should always be sorted by datetime. (see below the Flight class)

Methods:

fly(self, destination): Make the airplane take off and land if a flight is scheduled for this destination (see below the Flight class)

location_on_date(self, date): Returns where the plane will be on this date

available_on_date(self, date, location) : Returns True if the plane can fly from this location on this date (it can fly if it is in this location on this date and if it doesn’t already have a flight planned).

The Flight class

Attributes:

date : datetime.Date

destination : The destination airport. (see below the Airport class)

origin : The departure airport. (see below the Airport class)

plane : The plane used during this flight. (see above the Airplane class)

id (str) : The ID is an encoded string composed of the destination, the airlines code and the date.

Methods:

Those methods are here only to update the rest of the system:

take_off(self)

land(self) : change the location of the plane when it reaches its destination

The Airport class

Attributes:

city : (str) The code of the city where the airport is located

planes : The list of every plane that is currently in this airport. (see above the Airplane class)

scheduled_departures : The list of flight - Every future flight from this airport, sorted by date. (see above the Flight class)

scheduled_arrivals : The list of flight - Every future flight that will arrive to this airport, sorted by date. (see above the Flight class)

Methods:

schedule_flight(self, destination, datetime) :

finds an available airplane from an airline, that serves the departure and the destination

schedule the airplane for the flight

info(self, start_date, end_date) : Displays every scheduled flight from start_date to end_date.

You are free to add any class/method/attribute to your code, be sure to document everything you write.

Write a small code to test your program.

In [2]:
from datetime import date

class Airline:
    def __init__(self, id, name):
        self.id = id
        self.name = name
        self.planes = []

class Airplane:
    def __init__(self, id, current_location, company):
        self.id = id
        self.current_location = current_location
        self.company = company
        self.next_flights = []
        company.planes.append(self)
        current_location.planes.append(self)

    def fly(self, destination):
        if self.next_flights:
            flight = self.next_flights[0]
            if flight.destination == destination:
                flight.take_off()
                flight.land()
                self.next_flights.pop(0)

    def location_on_date(self, query_date):
        location = self.current_location
        for flight in self.next_flights:
            if flight.date == query_date:
                return flight.destination
            elif flight.date < query_date:
                location = flight.destination
        return location

    def available_on_date(self, query_date, location):
        return (self.location_on_date(query_date) == location and
                all(f.date != query_date for f in self.next_flights))

class Flight:
    def __init__(self, date, origin, destination, plane):
        self.date = date
        self.origin = origin
        self.destination = destination
        self.plane = plane
        self.id = f"{destination.city}-{plane.company.id}-{date.isoformat()}"
        plane.next_flights.append(self)
        plane.next_flights.sort(key=lambda f: f.date)
        origin.scheduled_departures.append(self)
        origin.scheduled_departures.sort(key=lambda f: f.date)
        destination.scheduled_arrivals.append(self)
        destination.scheduled_arrivals.sort(key=lambda f: f.date)

    def take_off(self):
        if self.plane in self.origin.planes:
            self.origin.planes.remove(self.plane)

    def land(self):
        self.destination.planes.append(self.plane)
        self.plane.current_location = self.destination

class Airport:
    def __init__(self, city):
        self.city = city
        self.planes = []
        self.scheduled_departures = []
        self.scheduled_arrivals = []

    def schedule_flight(self, destination, flight_date):
        for plane in self.planes:
            if plane.available_on_date(flight_date, self):
                Flight(flight_date, self, destination, plane)
                return f"Flight scheduled from {self.city} to {destination.city} on {flight_date}"
        return f"No available plane on {flight_date} from {self.city} to {destination.city}"

    def info(self, start_date, end_date):
        print(f"Flights from {self.city} between {start_date} and {end_date}:")
        for flight in self.scheduled_departures:
            if start_date <= flight.date <= end_date:
                print(f"  Flight {flight.id} to {flight.destination.city} on {flight.date}")


# Testing the Program
if __name__ == "__main__":
    # Creating airports
    jfk = Airport("JFK")
    lax = Airport("LAX")

    # Creating airline
    delta = Airline("DL", "Delta Airlines")

    # Creating airplanes
    plane1 = Airplane(101, jfk, delta)
    plane2 = Airplane(102, jfk, delta)

    # Scheduling flights
    print(jfk.schedule_flight(lax, date(2025, 4, 21)))
    print(jfk.schedule_flight(lax, date(2025, 4, 21)))
    print(jfk.schedule_flight(lax, date(2025, 4, 21)))

    # Information display
    jfk.info(date(2025, 4, 20), date(2025, 4, 22))

    # Fly plane
    plane1.fly(lax)
    print(f"Plane 101 is now in {plane1.current_location.city}")

Flight scheduled from JFK to LAX on 2025-04-21
Flight scheduled from JFK to LAX on 2025-04-21
No available plane on 2025-04-21 from JFK to LAX
Flights from JFK between 2025-04-20 and 2025-04-22:
  Flight LAX-DL-2025-04-21 to LAX on 2025-04-21
  Flight LAX-DL-2025-04-21 to LAX on 2025-04-21
Plane 101 is now in LAX
